# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 18 2022 11:16AM,248855,10,MSP214963,"Methapharm, Inc.",Released
1,Oct 18 2022 11:16AM,248855,10,MSP214965,"Methapharm, Inc.",Released
2,Oct 18 2022 11:16AM,248855,10,MSP214978,"Methapharm, Inc.",Released
3,Oct 18 2022 11:16AM,248855,10,MSP214968,"Methapharm, Inc.",Released
4,Oct 18 2022 11:16AM,248855,10,MSP214974,"Methapharm, Inc.",Released
5,Oct 18 2022 11:16AM,248855,10,MSP214975,"Methapharm, Inc.",Released
6,Oct 18 2022 11:16AM,248855,10,MSP214993,Methapharm-G,Released
7,Oct 18 2022 11:12AM,248854,10,0086085071,ISDIN Corporation,Released
8,Oct 18 2022 11:12AM,248854,10,0086085087,ISDIN Corporation,Released
9,Oct 18 2022 11:12AM,248854,10,0086086498,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,248851,Released,2
21,248852,Released,1
22,248853,Released,1
23,248854,Released,7
24,248855,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248851,NaN,2.0
248852,NaN,1.0
248853,NaN,1.0
248854,NaN,7.0
248855,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248636,0.0,1.0
248773,0.0,1.0
248802,0.0,31.0
248808,8.0,0.0
248813,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248636,0,1
248773,0,1
248802,0,31
248808,8,0
248813,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248636,0,1
1,248773,0,1
2,248802,0,31
3,248808,8,0
4,248813,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248636,,1
1,248773,,1
2,248802,,31
3,248808,8,
4,248813,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc."
6,Oct 18 2022 11:16AM,248855,10,Methapharm-G
7,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation
14,Oct 18 2022 11:05AM,248853,10,Bio-PRF
15,Oct 18 2022 10:56AM,248852,19,ACG North America LLC
16,Oct 18 2022 10:54AM,248851,19,VITABIOTICS USA INC.
18,Oct 18 2022 10:52AM,248850,19,VITABIOTICS USA INC.
19,Oct 18 2022 10:41AM,248849,20,"Exact-Rx, Inc."
22,Oct 18 2022 10:36AM,248848,19,VITABIOTICS USA INC.
23,Oct 18 2022 10:31AM,248844,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc.",,7
1,Oct 18 2022 11:16AM,248855,10,Methapharm-G,,7
2,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation,,7
3,Oct 18 2022 11:05AM,248853,10,Bio-PRF,,1
4,Oct 18 2022 10:56AM,248852,19,ACG North America LLC,,1
5,Oct 18 2022 10:54AM,248851,19,VITABIOTICS USA INC.,,2
6,Oct 18 2022 10:52AM,248850,19,VITABIOTICS USA INC.,,1
7,Oct 18 2022 10:41AM,248849,20,"Exact-Rx, Inc.",2,1
8,Oct 18 2022 10:36AM,248848,19,VITABIOTICS USA INC.,,1
9,Oct 18 2022 10:31AM,248844,20,"Exact-Rx, Inc.",11,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc.",7,
1,Oct 18 2022 11:16AM,248855,10,Methapharm-G,7,
2,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation,7,
3,Oct 18 2022 11:05AM,248853,10,Bio-PRF,1,
4,Oct 18 2022 10:56AM,248852,19,ACG North America LLC,1,
5,Oct 18 2022 10:54AM,248851,19,VITABIOTICS USA INC.,2,
6,Oct 18 2022 10:52AM,248850,19,VITABIOTICS USA INC.,1,
7,Oct 18 2022 10:41AM,248849,20,"Exact-Rx, Inc.",1,2
8,Oct 18 2022 10:36AM,248848,19,VITABIOTICS USA INC.,1,
9,Oct 18 2022 10:31AM,248844,20,"Exact-Rx, Inc.",2,11


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc.",7,
1,Oct 18 2022 11:16AM,248855,10,Methapharm-G,7,
2,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation,7,
3,Oct 18 2022 11:05AM,248853,10,Bio-PRF,1,
4,Oct 18 2022 10:56AM,248852,19,ACG North America LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc.",7.0,NaN
1,Oct 18 2022 11:16AM,248855,10,Methapharm-G,7.0,NaN
2,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation,7.0,NaN
3,Oct 18 2022 11:05AM,248853,10,Bio-PRF,1.0,NaN
4,Oct 18 2022 10:56AM,248852,19,ACG North America LLC,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 18 2022 11:16AM,248855,10,"Methapharm, Inc.",7.0,0.0
1,Oct 18 2022 11:16AM,248855,10,Methapharm-G,7.0,0.0
2,Oct 18 2022 11:12AM,248854,10,ISDIN Corporation,7.0,0.0
3,Oct 18 2022 11:05AM,248853,10,Bio-PRF,1.0,0.0
4,Oct 18 2022 10:56AM,248852,19,ACG North America LLC,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2239549,66.0,12.0
15,995364,68.0,35.0
16,248808,0.0,8.0
19,995401,5.0,0.0
20,746466,4.0,13.0
21,248636,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2239549,66.0,12.0
1,15,995364,68.0,35.0
2,16,248808,0.0,8.0
3,19,995401,5.0,0.0
4,20,746466,4.0,13.0
5,21,248636,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,66.0,12.0
1,15,68.0,35.0
2,16,0.0,8.0
3,19,5.0,0.0
4,20,4.0,13.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,66.0
1,15,Released,68.0
2,16,Released,0.0
3,19,Released,5.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,12.0,35.0,8.0,0.0,13.0,0.0
Released,66.0,68.0,0.0,5.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,12.0,35.0,8.0,0.0,13.0,0.0
1,Released,66.0,68.0,0.0,5.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,12.0,35.0,8.0,0.0,13.0,0.0
1,Released,66.0,68.0,0.0,5.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()